In [45]:
# this is extra images when a car will be rotated in a lane while shown a correct angle to straighten out



# Modifying previous preps notebook code to generate
# training images
# Note: this generates some black only images - they need to be deleted before training the model
# you can sort files by size - they will be all 3kb 
# Also you may need to run this with version 9.13 of Carla

#all imports
import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
sys.path.append('C:/CARLA_0.9.13/PythonAPI/carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner

SPEED_THRESHOLD = 2 #defines when we get close to desired speed so we drop the throttle

# Max steering angle
MAX_STEER_DEGREES = 40
#Max spin angle
MAX_SPIN = 20
# This is max actual angle with Mini under steering input=1.0
STEERING_CONVERSION = 75

#camera mount offset on the car - this mimics Tesla Model 3 view 
CAMERA_POS_Z = 1.3 
CAMERA_POS_X = 1.4 

# utility function for camera listening 
def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

# utility function for camera listening 
def sem_callback(image,data_dict):
    ########## IMPORTANT CHANGE for Semantic camera ##############
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function returns degrees between the car's direction 
    and direction to a selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = math.degrees(np.arctan2(y, x) - np.arctan2(car_vector.y, car_vector.x))
    # extra checks on predicted angle when values close to 360 degrees are returned
    if degrees<-180:
        degrees = degrees + 360
    elif degrees > 180:
        degrees = degrees - 360
    return degrees

def get_proper_angle(car,wp_idx,rte):
    '''
    This function uses simple fuction above to get angle but for current
    waypoint and a few more next waypoints to ensure we have not skipped
    next waypoint so we avoid the car trying to turn back
    '''
    # create a list of angles to next 5 waypoints starting with current
    next_angle_list = []
    for i in range(10):
        if wp_idx + i*3 <len(rte)-1:
            next_angle_list.append(get_angle(car,rte[wp_idx + i*3][0]))
    idx = 0
    while idx<len(next_angle_list)-2 and abs(next_angle_list[idx])>40:
        idx +=1
    return wp_idx+idx*3,next_angle_list[idx]  

def get_distant_angle(car,wp_idx,rte, delta):
    '''
    This function modifies the function above to get angle to a waypoint
    at a distance so we could use it for training image generation
    
    We will display the angle for now in the 'telemetry' view so
    we could play with how far forward we need to pick the waypoint
    '''
    if wp_idx + delta < len(rte)-1:
        i = wp_idx + delta
    else:
        i = len(rte)-1
    # check for intersection within the "look forward"
    # so we do not give turn results when just following the road
    intersection_detected = False
    for x in range(i-wp_idx):
        if rte[wp_idx+x][0].is_junction:
             intersection_detected = True
    angle = get_angle(car,rte[i][0])
    if not intersection_detected:
        result = 0
    elif angle <-10:
        result = -1
    elif angle>10:
        result =1
    else:
        result = 0    
    return result

def draw_route(wp, route,seconds=3.0):
    #draw the next few points route in sim window - Note it does not
    # get into the camera of the car
    if len(route)-wp <25: # route within 25 points from end is red
        draw_colour = carla.Color(r=255, g=0, b=0)
    else:
        draw_colour = carla.Color(r=0, g=0, b=255)
    for i in range(10):
        if wp+i<len(route)-2:
            world.debug.draw_string(route[wp+i][0].transform.location, '^', draw_shadow=False,
                color=draw_colour, life_time=seconds,
                persistent_lines=True)
    return None


def select_random_route(position,locs):
    '''
    retruns a random route for the car/veh
    out of the list of possible locations locs
    where distance is longer than 100 waypoints
    '''    
    point_a = position.location #we start at where the car is or last waypoint
    sampling_resolution = 1
    grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
    # now let' pick the longest possible route
    min_distance = 100
    result_route = None
    route_list = []
    for loc in locs: # we start trying all spawn points 
                                #but we just exclude first at zero index
        cur_route = grp.trace_route(point_a, loc.location)
        if len(cur_route) > min_distance:
            route_list.append(cur_route)
    result_route = random.choice(route_list)
    return result_route


# connect to the sim 
client = carla.Client('localhost', 2000)

time.sleep(5)
client.set_timeout(25)
#client.load_world('Town03')


# get world and spawn points
world = client.get_world()

# ensure sync mode on 
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.1
settings.no_rendering_mode = False
world.apply_settings(settings)

spawn_points = world.get_map().get_spawn_points()

#clean up any existing cars
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

#look for a blueprint of Tesla m3 car
vehicle_bp = world.get_blueprint_library().filter('*model3*')

def exit_clean():
    #clean up
    cv2.destroyAllWindows()
    camera_sem.stop()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    return None


#main loop
img_counter = 0
quit = False
while img_counter < 100:
    start_point = random.choice(spawn_points)
    vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
    time.sleep(2)
    # setting semantic camera
    camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
    camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.13 on Windows
    camera_bp.set_attribute('image_size_y', '480')
    camera_bp.set_attribute('fov', '90')
    camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
    camera_sem = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
    image_w = 640
    image_h = 480

    camera_data = {'sem_image': np.zeros((image_h,image_w,4)),
                   'rgb_image': np.zeros((image_h,image_w,4))}

        # this actually opens a live stream from the cameras
    camera_sem.listen(lambda image: sem_callback(image,camera_data))
    # adding collision sensor

    #setting RGB Camera 
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', '640') 
    camera_bp.set_attribute('image_size_y', '360')
    camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
    #this creates the camera in the sim
    camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
    image_w = camera_bp.get_attribute('image_size_x').as_int()
    image_h = camera_bp.get_attribute('image_size_y').as_int()
  
    # this actually opens a live stream from the camera
    camera.listen(lambda image: camera_callback(image,camera_data))
    cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
    cv2.imshow('RGB Camera',camera_data['rgb_image'])



    prev_position = vehicle.get_transform()
    # getting a random route for the car
    route = select_random_route(start_point,spawn_points)
    curr_wp = 0
    world.tick()
    
    while curr_wp<len(route)-6 and img_counter < 100:

        # move the car to next waypoint
        curr_wp +=1
        vehicle.set_transform(route[curr_wp][0].transform)
        time.sleep(2)
        world.tick()
                
        # first position the car on the route and take normal shot (without spinning)
        _, predicted_angle = get_proper_angle(vehicle,curr_wp+5,route)
        steer_input = predicted_angle
        # limit steering to max angel, say 40 degrees
        if predicted_angle<-MAX_STEER_DEGREES:
            steer_input = -MAX_STEER_DEGREES
        elif predicted_angle>MAX_STEER_DEGREES:
            steer_input = MAX_STEER_DEGREES
        gen_dir_angle = get_distant_angle(vehicle,curr_wp,route,30)
        initial_yaw = vehicle.get_transform().rotation.yaw
        sem_im = camera_data['sem_image']
        image = camera_data['rgb_image']
        img_counter += 1
        time_grab = time.time_ns()
        cv2.imwrite('_img/%06d_%s_%s.png' % (time_grab, gen_dir_angle,round(steer_input,0)), sem_im)
        cv2.imshow('RGB Camera',image)
        # only ouside intersections we spin the
        if route[curr_wp][0].is_intersection==False and route[curr_wp][0].is_junction==False:
            #grab images while spinning the car around
            for i in range(3):
                # Carla Tick
                trans = vehicle.get_transform()
                angle_adj = random.randrange(-MAX_SPIN, MAX_SPIN, 1)
                trans.rotation.yaw = initial_yaw +angle_adj 
                vehicle.set_transform(trans)
                world.tick()
                time.sleep(2)  #these delays seem to be necessary for the car to take the position before a shot is taken
                steer_input = predicted_angle - angle_adj # we put the opposite to correct back to straight
                if steer_input<-MAX_STEER_DEGREES:
                    steer_input = -MAX_STEER_DEGREES
                elif steer_input>MAX_STEER_DEGREES:
                    steer_input = MAX_STEER_DEGREES
                sem_im = camera_data['sem_image']
                img_counter += 1
                time_grab = time.time_ns()
                cv2.imwrite('_img/%06d_%s_%s.png' % (time_grab, gen_dir_angle,round(steer_input,0)), sem_im)
                image = camera_data['rgb_image']
                image = cv2.putText(image, 'Steer: '+str(steer_input), (30,30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.imshow('RGB Camera',image)
                if cv2.waitKey(0) == ord('q'):
                    quit = True
                    break
      
    if quit:
        break
exit_clean()



KeyboardInterrupt: 

In [28]:
# manual tweaks

client = carla.Client('localhost', 2000)

time.sleep(5)
client.set_timeout(25)
#client.load_world('Town03')


# get world and spawn points
world = client.get_world()

# ensure sync mode on 
settings = world.get_settings()
settings.synchronous_mode = False
world.apply_settings(settings)

spawn_points = world.get_map().get_spawn_points()

#clean up any existing cars
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

#look for a blueprint of Tesla m3 car
vehicle_bp = world.get_blueprint_library().filter('*model3*')


start_point = random.choice(spawn_points)
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
time.sleep(2)
# setting semantic camera
camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.13 on Windows
camera_bp.set_attribute('image_size_y', '480')
camera_bp.set_attribute('fov', '90')
camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
camera_sem = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
image_w = 640
image_h = 480

camera_data = {'sem_image': np.zeros((image_h,image_w,4)),
                'rgb_image': np.zeros((image_h,image_w,4))}

    # this actually opens a live stream from the cameras
camera_sem.listen(lambda image: sem_callback(image,camera_data))
# adding collision sensor

prev_position = vehicle.get_transform()
# getting a random route for the car
route = select_random_route(start_point,spawn_points)
curr_wp = 2

# move the car to next waypoint
curr_wp +=1
vehicle.set_transform(route[curr_wp][0].transform)
draw_route(4, route,seconds=60.0)

In [46]:
exit_clean()

In [32]:
_, predicted_angle = get_proper_angle(vehicle,curr_wp+5,route)
gen_dir_angle = get_distant_angle(vehicle,curr_wp,route,30)
initial_yaw = vehicle.get_transform().rotation.yaw
print(round(predicted_angle,0))
print(gen_dir_angle)
print(initial_yaw)

-12.0
-1
150.97317504882812


In [33]:
#test spinning
trans = vehicle.get_transform()
angle_adj = random.randrange(-MAX_STEER_DEGREES, MAX_STEER_DEGREES, 1)
trans.rotation.yaw = initial_yaw +angle_adj 
vehicle.set_transform(trans)
print(angle_adj)

33


In [42]:
curr_wp = 39
vehicle.set_transform(route[curr_wp][0].transform)
draw_route(curr_wp, route,seconds=10.0)
cur_point = route[curr_wp][0]
cur_point.is_intersection

True

In [34]:
draw_route(curr_wp+5, route,seconds=10.0)

In [8]:
#start from basics - working version with just RGB cam
import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
sys.path.append('C:/CARLA_0.9.13/PythonAPI/carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner

CAMERA_POS_Z = 1.3 
CAMERA_POS_X = 1.4 

CAM_HEIGHT = 480
CAM_WIDTH = 640
FOV = 90 # field of view = focal length

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]

def cleanup():
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for actor in world.get_actors().filter('*sensor*'):
        actor.destroy()

client = carla.Client('localhost', 2000)
time.sleep(5)
client.set_timeout(25)

# get world and spawn points
world = client.get_world()

# ensure sync mode on 
settings = world.get_settings()
settings.synchronous_mode = False
if settings.synchronous_mode:
    settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

cleanup()
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]
vehicle_bp = world.get_blueprint_library().filter('*model3*')
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
time.sleep(2)
vehicle.set_autopilot(True)

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
camera_bp.set_attribute('fov', str(FOV))
camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,3))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))


cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])

while True:
    # Display with imshow
    cv2.imshow('RGB Camera',camera_data['image'])
    
    # Break loop if user presses q
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()
cleanup()


In [25]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for actor in world.get_actors().filter('*sensor*'):
    actor.destroy()

In [26]:
cv2.destroyAllWindows()

In [13]:
# working version in Sync = False mode
# adding semantic segm camera to above
# this saves semantic images into _tst folder
import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
sys.path.append('C:/CARLA_0.9.13/PythonAPI/carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner

CAMERA_POS_Z = 1.3 
CAMERA_POS_X = 1.4 

CAM_HEIGHT = 480
CAM_WIDTH = 640
FOV = 90 # field of view = focal length

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]
def sem_callback(image,data_dict):
    ########## IMPORTANT CHANGE for Semantic camera ##############
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]

def cleanup():
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for actor in world.get_actors().filter('*sensor*'):
        actor.destroy()

client = carla.Client('localhost', 2000)
time.sleep(5)
client.set_timeout(25)

# get world and spawn points
world = client.get_world()

# ensure sync mode on 
settings = world.get_settings()
settings.synchronous_mode = False
if settings.synchronous_mode:
    settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

cleanup()
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]
vehicle_bp = world.get_blueprint_library().filter('*model3*')
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
time.sleep(2)
vehicle.set_autopilot(True)

#RGB CAM
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
camera_bp.set_attribute('fov', str(FOV))
camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()
camera_data = {'image': np.zeros((image_h,image_w,3)),
               'sem_image': np.zeros((image_h,image_w,3))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

#Semantgic cam
sem_camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
sem_camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
sem_camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
sem_camera_bp.set_attribute('fov', str(FOV))
#this creates the camera in the sim
sem_camera = world.spawn_actor(sem_camera_bp,camera_init_trans,attach_to=vehicle)
image_w = sem_camera_bp.get_attribute('image_size_x').as_int()
image_h = sem_camera_bp.get_attribute('image_size_y').as_int()
sem_camera_data = {'image': np.zeros((image_h,image_w,3))}
# this actually opens a live stream from the camera
#sem_camera.listen(lambda image: camera_callback(image,sem_camera_data))
sem_camera.listen(lambda image: sem_callback(image,camera_data))
time.sleep(2) #- this prevents black images but misses out on initial images
cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])

while True:
    # Display with imshow
    cv2.imshow('RGB Camera',camera_data['image'])
    time_grab = time.time_ns()
    cv2.imwrite('_tst/%06d.png' % (time_grab), camera_data['sem_image'])    
    # Break loop if user presses q
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()
cleanup()

: 

In [11]:
# try above in Synchronous mode
# this works
import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
sys.path.append('C:/CARLA_0.9.13/PythonAPI/carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner

CAMERA_POS_Z = 1.3 
CAMERA_POS_X = 1.4 

CAM_HEIGHT = 480
CAM_WIDTH = 640
FOV = 90 # field of view = focal length

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]
def sem_callback(image,data_dict):
    ########## IMPORTANT CHANGE for Semantic camera ##############
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]

def cleanup():
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for actor in world.get_actors().filter('*sensor*'):
        actor.destroy()

client = carla.Client('localhost', 2000)
time.sleep(5)
client.set_timeout(25)

# get world and spawn points
world = client.get_world()

# ensure sync mode on 
settings = world.get_settings()
settings.synchronous_mode = True
if settings.synchronous_mode:
    settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

cleanup()
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]
vehicle_bp = world.get_blueprint_library().filter('*model3*')
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
time.sleep(2)
vehicle.set_autopilot(True)

#RGB CAM
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
camera_bp.set_attribute('fov', str(FOV))
camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()
camera_data = {'image': np.zeros((image_h,image_w,3)),
               'sem_image': np.zeros((image_h,image_w,3))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

#Semantgic cam
sem_camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
sem_camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
sem_camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
sem_camera_bp.set_attribute('fov', str(FOV))
#this creates the camera in the sim
sem_camera = world.spawn_actor(sem_camera_bp,camera_init_trans,attach_to=vehicle)
image_w = sem_camera_bp.get_attribute('image_size_x').as_int()
image_h = sem_camera_bp.get_attribute('image_size_y').as_int()
sem_camera_data = {'image': np.zeros((image_h,image_w,3))}
# this actually opens a live stream from the camera
#sem_camera.listen(lambda image: camera_callback(image,sem_camera_data))
sem_camera.listen(lambda image: sem_callback(image,camera_data))

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])

while True:
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        break
    # Display with imshow
    cv2.imshow('RGB Camera',camera_data['image'])
    time_grab = time.time_ns()
    sem_image = camera_data['sem_image']
    if np.sum(sem_image) > 0:   #check for black images
        cv2.imwrite('_tst/%06d.png' % (time_grab), sem_image)    
    # Break loop if user presses q
cv2.destroyAllWindows()
cleanup()

In [49]:
# add navigation loops to above
# This is final working version, but needs to be interrupted manually
# when you think you got enough images

import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
sys.path.append('C:/CARLA_0.9.13/PythonAPI/carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner

CAMERA_POS_Z = 1.3 
CAMERA_POS_X = 1.4 

CAM_HEIGHT = 480
CAM_WIDTH = 640
FOV = 90 # field of view = focal length

YAW_ADJ_DEGREES = 25 #random spin angle max

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]
def sem_callback(image,data_dict):
    ########## IMPORTANT CHANGE for Semantic camera ##############
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]

def cleanup():
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for actor in world.get_actors().filter('*sensor*'):
        actor.destroy()

# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function returns degrees between the car's direction 
    and direction to a selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    # vector to waypoint
    if ((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5 == 0:
        x=0
        y=0
    else:
        x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
        y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
        
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = math.degrees(np.arctan2(y, x) - np.arctan2(car_vector.y, car_vector.x))
    # extra checks on predicted angle when values close to 360 degrees are returned
    if degrees<-180:
        degrees = degrees + 360
    elif degrees > 180:
        degrees = degrees - 360
    return degrees


def get_distant_angle(car,wp_idx,rte, delta):
    '''
    This function modifies the function above to get angle to a waypoint
    at a distance so we could use it for training image generation
    
    We will display the angle for now in the 'telemetry' view so
    we could play with how far forward we need to pick the waypoint
    '''
    if wp_idx + delta < len(rte)-1:
        i = wp_idx + delta
    else:
        i = len(rte)-1
    # check for intersection within the "look forward"
    # so we do not give turn results when just following the road
    intersection_detected = False
    for x in range(i-wp_idx):
        if rte[wp_idx+x][0].is_junction:
             intersection_detected = True
    angle = get_angle(car,rte[i][0])
    if not intersection_detected:
        result = 0
    elif angle <-10:
        result = -1
    elif angle>10:
        result =1
    else:
        result = 0    
    return result

def draw_route(wp, route,seconds=3.0):
    #draw the next few points route in sim window - Note it does not
    # get into the camera of the car
    if len(route)-wp <25: # route within 25 points from end is red
        draw_colour = carla.Color(r=255, g=0, b=0)
    else:
        draw_colour = carla.Color(r=0, g=0, b=255)
    for i in range(10):
        if wp+i<len(route)-2:
            world.debug.draw_string(route[wp+i][0].transform.location, '^', draw_shadow=False,
                color=draw_colour, life_time=seconds,
                persistent_lines=True)
    return None


def select_random_route(position,locs):
    '''
    retruns a random route for the car/veh
    out of the list of possible locations locs
    where distance is longer than 100 waypoints
    '''    
    point_a = position.location #we start at where the car is or last waypoint
    sampling_resolution = 1
    grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
    # now let' pick the longest possible route
    min_distance = 100
    result_route = None
    route_list = []
    for loc in locs: # we start trying all spawn points 
                                #but we just exclude first at zero index
        cur_route = grp.trace_route(point_a, loc.location)
        if len(cur_route) > min_distance:
            route_list.append(cur_route)
    result_route = random.choice(route_list)
    return result_route

client = carla.Client('localhost', 2000)
time.sleep(5)
client.set_timeout(25)

# get world and spawn points
world = client.get_world()

 
settings = world.get_settings()
settings.synchronous_mode = False
if settings.synchronous_mode:
    settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

cleanup()
spawn_points = world.get_map().get_spawn_points()

quit = False
while not quit:
    start_point = random.choice(spawn_points)
    vehicle_bp = world.get_blueprint_library().filter('*model3*')
    vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
    time.sleep(2)
 
    #RGB CAM
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
    camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
    camera_bp.set_attribute('fov', str(FOV))
    camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
    #this creates the camera in the sim
    camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
    image_w = camera_bp.get_attribute('image_size_x').as_int()
    image_h = camera_bp.get_attribute('image_size_y').as_int()
    camera_data = {'image': np.zeros((image_h,image_w,3)),
                'sem_image': np.zeros((image_h,image_w,3))}
    # this actually opens a live stream from the camera
    camera.listen(lambda image: camera_callback(image,camera_data))

    #Semantgic cam
    sem_camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
    sem_camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
    sem_camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
    sem_camera_bp.set_attribute('fov', str(FOV))
    #this creates the camera in the sim
    sem_camera = world.spawn_actor(sem_camera_bp,camera_init_trans,attach_to=vehicle)
    image_w = sem_camera_bp.get_attribute('image_size_x').as_int()
    image_h = sem_camera_bp.get_attribute('image_size_y').as_int()
    sem_camera_data = {'image': np.zeros((image_h,image_w,3))}
    # this actually opens a live stream from the camera
    #sem_camera.listen(lambda image: camera_callback(image,sem_camera_data))
    sem_camera.listen(lambda image: sem_callback(image,camera_data))

    cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
    cv2.imshow('RGB Camera',camera_data['image'])
    #make a route
    route = select_random_route(start_point,spawn_points)
    for idx, waypoint in enumerate(route): # move the car through the route
        #world.tick()
        transform = waypoint[0].transform
        vehicle.set_transform(transform)
        vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
        time.sleep(2) #these delays seem to be necessary for teh car to take the position before a shot is taken
        initial_yaw = waypoint[0].transform.rotation.yaw
        
        for i in range(5):
            #world.tick()         
            trans = waypoint[0].transform
            angle_adj = random.randrange(-YAW_ADJ_DEGREES, YAW_ADJ_DEGREES, 1)
            trans.rotation.yaw = initial_yaw +angle_adj 
            vehicle.set_transform(trans)
            vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
            time.sleep(1)  #these delays seem to be necessary for the car to take the position before a shot is taken
            gen_dir_angle = get_distant_angle(vehicle,idx,route,30) # general angle taken before spinning the car
            if cv2.waitKey(1) == ord('q'):
                quit = True
                break
            # Display with imshow
            cv2.imshow('RGB Camera',camera_data['image'])
            # save semantic image watching out for end of route    
            if idx +5 < len(route)-1:
                predicted_angle = get_angle(vehicle,route[idx+5][0]) # we always get the angle to +5 waypoint ahead of us
                
                time_grab = time.time_ns()
                sem_image = camera_data['sem_image']
                if np.sum(sem_image) > 0:   #check for black images
                    cv2.imwrite('_img/%06d_%s_%s.png' % (time_grab, gen_dir_angle,round(predicted_angle,0)), sem_image)
    cleanup()
    if quit:
        break
    # Break loop if user presses q
cv2.destroyAllWindows()
cleanup()


KeyboardInterrupt: 

In [1]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for actor in world.get_actors().filter('*sensor*'):
    actor.destroy()
cv2.destroyAllWindows()

NameError: name 'world' is not defined

In [5]:
# adding Checks for lane changes 
# images from waypoints before lane changes should be IGNORED
# as they have "sudden" unexplained

# This is a working version, but needs to be interrupted manually
# when you think you got enough images

import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
sys.path.append('C:/CARLA_0.9.13/PythonAPI/carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner

CAMERA_POS_Z = 1.3 
CAMERA_POS_X = 1.4 

CAM_HEIGHT = 480
CAM_WIDTH = 640
FOV = 90 # field of view = focal length

YAW_ADJ_DEGREES = 25 #random spin angle max

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]
def sem_callback(image,data_dict):
    ########## IMPORTANT CHANGE for Semantic camera ##############
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]

def cleanup():
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for actor in world.get_actors().filter('*sensor*'):
        actor.destroy()

# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function returns degrees between the car's direction 
    and direction to a selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    # vector to waypoint
    if ((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5 == 0:
        x=0
        y=0
    else:
        x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
        y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
        
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = math.degrees(np.arctan2(y, x) - np.arctan2(car_vector.y, car_vector.x))
    # extra checks on predicted angle when values close to 360 degrees are returned
    if degrees<-180:
        degrees = degrees + 360
    elif degrees > 180:
        degrees = degrees - 360
    return degrees


def get_distant_angle(car,wp_idx,rte, delta):
    '''
    This function modifies the function above to get angle to a waypoint
    at a distance so we could use it for training image generation
    
    We will display the angle for now in the 'telemetry' view so
    we could play with how far forward we need to pick the waypoint
    '''
    if wp_idx + delta < len(rte)-1:
        i = wp_idx + delta
    else:
        i = len(rte)-1
    # check for intersection within the "look forward"
    # so we do not give turn results when just following the road
    intersection_detected = False
    for x in range(i-wp_idx):
        if rte[wp_idx+x][0].is_junction or rte[wp_idx+x][0].is_intersection:
             intersection_detected = True
    angle = get_angle(car,rte[i][0])
    if not intersection_detected:
        result = 0
    elif angle <-10:
        result = -1
    elif angle>10:
        result =1
    else:
        result = 0    
    return result

def draw_route(wp, route,seconds=5.0):
    #draw the next few points route in sim window - Note it does not
    # get into the camera of the car
    if len(route)-wp <25: # route within 25 points from end is red
        draw_colour = carla.Color(r=255, g=0, b=0)
    else:
        draw_colour = carla.Color(r=0, g=0, b=255)
    for i in range(15):
        if wp+i<len(route)-2:
            world.debug.draw_string(route[wp+i][0].transform.location, '^', draw_shadow=False,
                color=draw_colour, life_time=seconds,
                persistent_lines=True)
    return None


def select_random_route(position,locs):
    '''
    retruns a random route for the car/veh
    out of the list of possible locations locs
    where distance is longer than 100 waypoints
    '''    
    point_a = position.location #we start at where the car is or last waypoint
    sampling_resolution = 1
    grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
    # now let' pick the longest possible route
    min_distance = 100
    result_route = None
    route_list = []
    for loc in locs: # we start trying all spawn points 
                                #but we just exclude first at zero index
        cur_route = grp.trace_route(point_a, loc.location)
        if len(cur_route) > min_distance:
            route_list.append(cur_route)
    result_route = random.choice(route_list)
    return result_route

client = carla.Client('localhost', 2000)
time.sleep(5)
client.set_timeout(25)

# get world and spawn points
world = client.get_world()

 
settings = world.get_settings()
settings.synchronous_mode = False
if settings.synchronous_mode:
    settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

cleanup()
spawn_points = world.get_map().get_spawn_points()

quit = False
while not quit:
    start_point = random.choice(spawn_points)
    vehicle_bp = world.get_blueprint_library().filter('*model3*')
    vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
    time.sleep(2)
 
    #RGB CAM
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
    camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
    camera_bp.set_attribute('fov', str(FOV))
    camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
    #this creates the camera in the sim
    camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
    image_w = camera_bp.get_attribute('image_size_x').as_int()
    image_h = camera_bp.get_attribute('image_size_y').as_int()
    camera_data = {'image': np.zeros((image_h,image_w,3)),
                'sem_image': np.zeros((image_h,image_w,3))}
    # this actually opens a live stream from the camera
    camera.listen(lambda image: camera_callback(image,camera_data))

    #Semantgic cam
    sem_camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
    sem_camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
    sem_camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
    sem_camera_bp.set_attribute('fov', str(FOV))
    #this creates the camera in the sim
    sem_camera = world.spawn_actor(sem_camera_bp,camera_init_trans,attach_to=vehicle)
    image_w = sem_camera_bp.get_attribute('image_size_x').as_int()
    image_h = sem_camera_bp.get_attribute('image_size_y').as_int()
    sem_camera_data = {'image': np.zeros((image_h,image_w,3))}
    # this actually opens a live stream from the camera
    #sem_camera.listen(lambda image: camera_callback(image,sem_camera_data))
    sem_camera.listen(lambda image: sem_callback(image,camera_data))

    cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
    cv2.imshow('RGB Camera',camera_data['image'])
    #make a route
    route = select_random_route(start_point,spawn_points)
    gen_dir_angle = 0 # in case we did not get a general GPS direction
    for idx, waypoint in enumerate(route): # move the car through the route
        
        transform = waypoint[0].transform
        vehicle.set_transform(transform)
        vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
        time.sleep(2) #these delays seem to be necessary for teh car to take the position before a shot is taken
        initial_yaw = waypoint[0].transform.rotation.yaw
        # GPS general direction is only taken outside intersections
        # so that direction is retained throughout the same intersection until it is finished
        if not waypoint[0].is_intersection and not waypoint[0].is_junction:
            gen_dir_angle = get_distant_angle(vehicle,idx,route,30) # general angle taken before spinning the car
        # logic to detect a lane change and ignore/not take those images
        draw_route(idx, route,seconds=5.0)
        lane_change = False
        if not waypoint[0].is_intersection and not waypoint[0].is_junction:
            if idx < len(route)-2:
                if route[idx][0].lane_id != route[idx+1][0].lane_id:
                    lane_change = True
        if not lane_change:
            for i in range(5):
                trans = waypoint[0].transform
                angle_adj = random.randrange(-YAW_ADJ_DEGREES, YAW_ADJ_DEGREES, 1)
                trans.rotation.yaw = initial_yaw +angle_adj 
                vehicle.set_transform(trans)
                vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
                time.sleep(1)  #these delays seem to be necessary for the car to take the position before a shot is taken
                
                if cv2.waitKey(1) == ord('q'):
                    quit = True
                    break
                # Display with imshow
                cv2.imshow('RGB Camera',camera_data['image'])
                # save semantic image watching out for end of route    
                if idx +5 < len(route)-1:
                    predicted_angle = get_angle(vehicle,route[idx+5][0]) # we always get the angle to +5 waypoint ahead of us
                    
                    time_grab = time.time_ns()
                    sem_image = camera_data['sem_image']
                    if np.sum(sem_image) > 0:   #check for black images
                        cv2.imwrite('_img/%06d_%s_%s.png' % (time_grab, gen_dir_angle,round(predicted_angle,0)), sem_image)
    cleanup()
    if quit:
        break
    # Break loop if user presses q
cv2.destroyAllWindows()
cleanup()


KeyboardInterrupt: 

In [3]:
cv2.destroyAllWindows()
cleanup()

In [4]:
# cleaning direction function
# right and left can be done only at upcoming intersections
# or at lane changes

# This is a working version, but needs to be interrupted manually
# when you think you got enough images

import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
sys.path.append('C:/CARLA_0.9.13/PythonAPI/carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner


SHOW_RGB = False

CAMERA_POS_Z = 1.3 
CAMERA_POS_X = 1.4 

CAM_HEIGHT = 480
CAM_WIDTH = 640
FOV = 90 # field of view = focal length

YAW_ADJ_DEGREES = 15 #random spin angle max

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]
def sem_callback(image,data_dict):
    ########## IMPORTANT CHANGE for Semantic camera ##############
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]

def cleanup():
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for actor in world.get_actors().filter('*sensor*'):
        actor.destroy()

# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function returns degrees between the car's direction 
    and direction to a selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    # vector to waypoint
    if ((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5 == 0:
        x=0
        y=0
    else:
        x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
        y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
        
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = math.degrees(np.arctan2(y, x) - np.arctan2(car_vector.y, car_vector.x))
    # extra checks on predicted angle when values close to 360 degrees are returned
    if degrees<-180:
        degrees = degrees + 360
    elif degrees > 180:
        degrees = degrees - 360
    return degrees


#latest get direction function
def get_distant_angle(car,wp_idx,rte, delta):
    '''
    This function determines general direction
    given what we are planning to do at an oncoming intersection
    so if we are within delta points from an intersection
    we check our route at the intersection to decide what we do at it
    '''
    if wp_idx + delta < len(rte)-1:
        i = wp_idx + delta
    else:
        i = len(rte)-1
    # check for intersection within the "look forward"
    # so we do not give turn results when just following the road
    intersection_detected = False
    for x in range(i-wp_idx):
        if rte[wp_idx+x][0].is_junction:
             intersection_detected = True
             intersection_ref = wp_idx+x
             break
    if not intersection_detected:
        result = 0
    else: #we check out the intersection
        angles_planned = [] # this is a list of angles towards current exit from the intersection
        all_angles = []
        #this is how to get all waypoints defining an intersection
        junction_wps = rte[intersection_ref][0].get_junction().get_waypoints(carla.LaneType.Driving)
        for wp in junction_wps:
            angle = int(get_angle(car,wp[1])) 
            #this 'if' below excludes close exits points to entry - exits for which you would need to take u-turns so they do not count
            if wp[1].transform.location.distance(route[intersection_ref][0].transform.location) > 20: 
                # check all exits for proximity to our route so we can flag the exit we are planning to take
                for i in range(intersection_ref,len(route)-1):
                    if wp[1].transform.location.distance(route[i][0].transform.location) < 10:
                        angles_planned.append(angle)
                    else:
                        all_angles.append(angle)
        angles_planned = list(set(angles_planned))
        all_angles = list(set(all_angles))
        alternative_angles = [item for item in all_angles if item not in angles_planned] 
        if len(alternative_angles) == 0 or len(angles_planned) == 0:
            result = 0
        elif min(angles_planned)<-25 and (min(alternative_angles) > min(angles_planned)):
            #we are planningleft turn
            result = -1
        elif max(angles_planned)>25 and (max(alternative_angles) < max(angles_planned)):
            result = 1
        else:
            result = 0  
    return result

def draw_route(wp, route,seconds=5.0):
    #draw the next few points route in sim window - Note it does not
    # get into the camera of the car
    if len(route)-wp <25: # route within 25 points from end is red
        draw_colour = carla.Color(r=255, g=0, b=0)
    else:
        draw_colour = carla.Color(r=0, g=0, b=255)
    for i in range(50):
        if wp+i<len(route)-2:
            world.debug.draw_string(route[wp+i][0].transform.location, '^', draw_shadow=False,
                color=draw_colour, life_time=seconds,
                persistent_lines=True)
    return None


def select_random_route(position,locs):
    '''
    retruns a random route for the car/veh
    out of the list of possible locations locs
    where distance is longer than 100 waypoints
    '''    
    point_a = position.location #we start at where the car is or last waypoint
    sampling_resolution = 1
    grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
    # now let' pick the longest possible route
    min_distance = 100
    result_route = None
    route_list = []
    for loc in locs: # we start trying all spawn points 
                                #but we just exclude first at zero index
        cur_route = grp.trace_route(point_a, loc.location)
        if len(cur_route) > min_distance:
            route_list.append(cur_route)
    result_route = random.choice(route_list)
    return result_route

client = carla.Client('localhost', 2000)
time.sleep(5)
client.set_timeout(25)

# get world and spawn points
world = client.get_world()

 
settings = world.get_settings()
settings.synchronous_mode = False
settings.no_rendering_mode = True
if settings.synchronous_mode:
    settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

cleanup()
spawn_points = world.get_map().get_spawn_points()

quit = False
while not quit:
    start_point = random.choice(spawn_points)
    vehicle_bp = world.get_blueprint_library().filter('*model3*')
    vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
    time.sleep(2)
 
    #RGB CAM
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
    camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
    camera_bp.set_attribute('fov', str(FOV))
    camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
    #this creates the camera in the sim
    camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
    image_w = camera_bp.get_attribute('image_size_x').as_int()
    image_h = camera_bp.get_attribute('image_size_y').as_int()
    camera_data = {'image': np.zeros((image_h,image_w,3)),
                'sem_image': np.zeros((image_h,image_w,3))}
    # this actually opens a live stream from the camera
    camera.listen(lambda image: camera_callback(image,camera_data))

    #Semantgic cam
    sem_camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
    sem_camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
    sem_camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
    sem_camera_bp.set_attribute('fov', str(FOV))
    #this creates the camera in the sim
    sem_camera = world.spawn_actor(sem_camera_bp,camera_init_trans,attach_to=vehicle)
    image_w = sem_camera_bp.get_attribute('image_size_x').as_int()
    image_h = sem_camera_bp.get_attribute('image_size_y').as_int()
    sem_camera_data = {'image': np.zeros((image_h,image_w,3))}
    # this actually opens a live stream from the camera
    #sem_camera.listen(lambda image: camera_callback(image,sem_camera_data))
    sem_camera.listen(lambda image: sem_callback(image,camera_data))

    if SHOW_RGB:
        cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RGB Camera',camera_data['image'])
    #make a route
    route = select_random_route(start_point,spawn_points)
    gen_dir_angle = 0 # in case we did not get a general GPS direction
    for idx, waypoint in enumerate(route): # move the car through the route
        
        transform = waypoint[0].transform
        vehicle.set_transform(transform)
        vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
        time.sleep(2) #these delays seem to be necessary for teh car to take the position before a shot is taken
        initial_yaw = waypoint[0].transform.rotation.yaw
        # GPS general direction is only taken outside intersections
        # so that direction is retained throughout the same intersection until it is finished
        if not waypoint[0].is_intersection and not waypoint[0].is_junction:
            gen_dir_angle = get_distant_angle(vehicle,idx,route,30) # general angle taken before spinning the car
        # logic to detect a lane change and ignore/not take those images
        draw_route(idx, route,seconds=5.0)
        lane_change = False
        if not waypoint[0].is_intersection and not waypoint[0].is_junction:
            if idx < len(route)-2:
                if route[idx][0].lane_id != route[idx+1][0].lane_id:
                    lane_change = True
        if lane_change: # lane changes are treated as turns in general direction
            if get_angle(vehicle,route[idx+1][0])<0:
                gen_dir_angle = -1
            else:
                gen_dir_angle = 1

        for i in range(5):
            trans = waypoint[0].transform
            angle_adj = random.randrange(-YAW_ADJ_DEGREES, YAW_ADJ_DEGREES, 1)
            trans.rotation.yaw = initial_yaw +angle_adj 
            vehicle.set_transform(trans)
            vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
            time.sleep(1)  #these delays seem to be necessary for the car to take the position before a shot is taken
            
            # Display with imshow
            if SHOW_RGB:
                cv2.imshow('RGB Camera',camera_data['image'])
            # save semantic image watching out for end of route    
            if idx +5 < len(route)-1:
                predicted_angle = get_angle(vehicle,route[idx+5][0]) # we always get the angle to +5 waypoint ahead of us
                
                time_grab = time.time_ns()
                sem_image = camera_data['sem_image']
                if np.sum(sem_image) > 0:   #check for black images
                    cv2.imwrite('_img/%06d_%s_%s.png' % (time_grab, gen_dir_angle,round(predicted_angle,0)), sem_image)
    cleanup()
    if quit:
        break
    # Break loop if user presses q
cv2.destroyAllWindows()
cleanup()

KeyboardInterrupt: 